In [17]:
import requests
import pandas as pd
import glob

In [19]:
# récupérer tous les fichiers du dossier videos et dédupliquer les lignes
files = glob.glob("CSV_Categories/*.csv")

df = []
for f in files:
    csv = pd.read_csv(f)
    df.append(csv)
df = pd.concat(df,ignore_index=True)
df.drop_duplicates(subset=['video_id'], keep="first", inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13794 entries, 0 to 13830
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   video_id               13794 non-null  object 
 1   title                  13794 non-null  object 
 2   description            9156 non-null   object 
 3   channel_id             13794 non-null  object 
 4   category_id            13794 non-null  int64  
 5   language               13794 non-null  object 
 6   channel                13794 non-null  object 
 7   published_at           13794 non-null  object 
 8   duration               13778 non-null  object 
 9   views                  13794 non-null  int64  
 10  likes                  13794 non-null  int64  
 11  comments               13794 non-null  int64  
 12  Engagement total       13794 non-null  int64  
 13  Taux d'engagement (%)  13794 non-null  float64
 14  duration_td            13778 non-null  object 
 15  Durée (

In [21]:
# langues: supprimer les lignes avec valeurs
df.dropna(subset="language",axis=0,inplace=True)

# langues: regrouper fr_fr, be_fr sous 'fr', pareil pour 'en'
df.loc[df['language'].str.contains('^fr'),'language'] = "fr"
df.loc[df['language'].str.contains('^en'),'language'] = "en"

In [23]:
#supprimer autres langues
df['language'].value_counts()

language
fr       8062
en       3268
hi        814
ja        171
de        123
         ... 
zh-CN       1
lb          1
ba          1
sk          1
de-AT       1
Name: count, Length: 77, dtype: int64

In [25]:
# calculer le taux d'engagement pour voir en un coup d'oeil les vidéos ayant reçu le plus d'intéractions
df["Engagement total"] = df["likes"] + df["comments"]
df["Taux d'engagement (%)"] = (df["Engagement total"] / df["views"].replace(0, 1)) * 100

In [27]:
# transofrmer les durées (ici min/sec) en secondes (plus simple pour l'analyse
import re

# 1. Conversion en timedelta (errors='coerce' transforme les formats invalides en NaT)
df['duration_td'] = pd.to_timedelta(df['duration'], errors='coerce')

# 2. Conversion en secondes
# .astype('Int64') permet de garder les NaN si nécessaire sans faire planter le code
df['Durée (s)'] = df['duration_td'].dt.total_seconds().astype('Int64')

# OPTIONNEL : Si vous voulez remplacer les vides par 0 et avoir des entiers classiques
# df['Durée (s)'] = df['duration_td'].dt.total_seconds().fillna(0).astype(int)

print(df[['duration', 'Durée (s)']])

       duration  Durée (s)
0         PT11S         11
1         PT26S         26
2      PT10M29S        629
3         PT58S         58
4         PT17S         17
...         ...        ...
13826     PT43S         43
13827   PT9M24S        564
13828     PT25S         25
13829  PT38M44S       2324
13830      PT8S          8

[13794 rows x 2 columns]


In [29]:
# colonnes avec valeurs nulls
df.isna().sum()

video_id                    0
title                       0
description              4638
channel_id                  0
category_id                 0
language                    0
channel                     0
published_at                0
duration                   16
views                       0
likes                       0
comments                    0
Engagement total            0
Taux d'engagement (%)       0
duration_td                16
Durée (s)                  16
Unnamed: 0               5775
dtype: int64

In [31]:
df.shape

(13794, 17)

In [35]:
df

,video_id,title,description,channel_id,category_id,language,channel,published_at,duration,views,likes,comments,Engagement total,Taux d'engagement (%),duration_td,Durée (s),Unnamed: 0
0,eVrbPNIHfHY,LES JEUNES EN SALLE DE SPORT💪🤣#humour #shorts,NaN,UCpsvoU1qL4WLR6jvh1CWjhw,22,fr,Le déraciné,2025-06-16T16:01:04Z,PT11S,2640830,49359,450,49809,1.886112,0 days 00:00:11,11,NaN
1,kMPxRclSRrE,Je ne suis qu’un homme!!! #humour #lol #love #...,NaN,UCcIk1sZMBTu91ZxUHLr-BOg,23,fr,Kévin Lévy,2025-12-14T16:31:40Z,PT26S,919082,8487,144,8631,0.939089,0 days 00:00:26,26,NaN
2,E9CnEeOdESk,Sacha Ferra – C'est son style,Sacha Ferra – C'est son style\n\n𝙀𝙉𝙑𝙄𝙀 𝘿'𝙀𝙉𝘾𝙊𝙍...,UCPHJfLg1VeNBsfzzTszpmmQ,23,fr,Montreux Comedy,2025-10-08T16:00:32Z,PT10M29S,143065,4670,81,4751,3.320868,0 days 00:10:29,629,NaN
3,9qnIkeqYWtE,Por favor comparte con tus amigos #pov #herram...,NaN,UCUnw7mc6iHy8vZe8mvMKlUA,22,en,Mai Nguyên,2026-01-22T10:03:17Z,PT58S,1522,47,15,62,4.073587,0 days 00:00:58,58,NaN
4,hTKWX0cQtiE,Le dialogue le plus WTF jamais entendu ? #humo...,"C'est un papier qui tombe à l'eau. Et il crie,...",UCIQs0qjGOmjV2jTkazCvXyQ,22,fr,comedyUp,2025-12-20T22:08:50Z,PT17S,523750,2404,3,2407,0.459570,0 days 00:00:17,17,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13826,vlT1LGtvM5o,Routine for flexibility now in my bio ✅ #footb...,NaN,UCR5oDhM18tC-xWhZo_EapOw,17,en,Kivenro,2025-06-14T21:01:00Z,PT43S,8798665,128057,352,128409,1.459415,0 days 00:00:43,43,NaN
13827,V5IeC3ntcSg,Les transformations chimiques - cours de secon...,📈Les transformations chimiques pour la classe ...,UCmyeFnJJZE7mCtvbQ8IJitQ,27,fr,Les génies des sciences,2023-01-10T18:30:26Z,PT9M24S,139099,3538,126,3664,2.634095,0 days 00:09:24,564,NaN
13828,Qkaco2atfQo,Tone your lower body with this full leg workou...,NaN,UCo1bpycul1Xhpbo3gkhBBww,22,en,CallmeUche,2024-04-09T22:30:37Z,PT25S,2433500,89749,881,90630,3.724265,0 days 00:00:25,25,NaN
13829,ihUUzigRlDY,LES SECRETS de la TRANSFORMATION PHYSIQUE et M...,OBJECTIF RING. Épisode 2 : La Métamorphose.\n\...,UCwVnmWM7fmu6fTZd0AMzufg,26,fr,Training Thérapie,2024-12-01T17:58:17Z,PT38M44S,131126,4729,131,4860,3.706359,0 days 00:38:44,2324,NaN
